In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from utils import *

In [3]:
now = datetime.utcnow()
round_down_6h = (now - timedelta(hours=now.hour % 6)).replace(minute=0, second=0, microsecond=0)

In [4]:
start = round_down_6h
end = start + timedelta(minutes=10)

In [5]:
start

datetime.datetime(2020, 9, 25, 18, 0)

In [72]:
G16_DIR = G16_ROOT / start.strftime('%Y%m%d/L1b/G16/RadF')
G17_DIR = G17_ROOT / start.strftime('%Y%m%d/L1b/G17/RadF')
H8_DIR = H8_ROOT / start.strftime('%Y/%Y_%m_%d_%j/%H%M')
for d in G16_DIR, G17_DIR, H8_DIR:
    assert d.is_dir(), d

In [9]:
L1B_DIR = Path('l1b')
L1B_DIR.mkdir(exist_ok=True)

In [79]:
def get_or_files(d):
    files = pd.Series({parse_or(f.name):f for f in d.glob('*.nc')}).sort_index()
    files.index.names=['start','end','band']
    return files

def get_dat_files(d):
    files = {}
    for f in d.rglob("*FLDK*.DAT"):
        files[parse_dat(f.name)] = f
    files = pd.Series(files).sort_index()
    files.index.names=['start','band','segment']
    return files

In [86]:
def get_fd_goes(files, start, end):
    candidates = files.loc[:end, start:]
    longest = candidates.groupby(['start','band']).apply(lambda x: x.sort_index().iloc[-1])
    sizes = longest.groupby('start').size()
    full = longest.loc[[sizes.index[sizes == 16][0]]]
    return full

In [107]:
def get_fd_him(files, start):
    all_segments = [f'S{i:02}10' for i in range(1,11)]
    files = files.loc[start]
    def all_or_none(x):
        segs = x.droplevel('band').reindex(all_segments)
        if segs.isnull().any():
            return []
        else:
            return segs
    full_files = files.groupby(['band']).apply(all_or_none)
    assert set(files.index.get_level_values('band').unique()) == set(range(1,17))
    return full_files

In [109]:
import shutil

In [110]:
def save_goes_files(out_root, files):
    with tqdm(list(files.iteritems())) as bar:
        for (start, band), f in bar:
            out_dir = out_root / start.strftime('%Y%m%dT%H%M') / f'{band:02d}'
            out_dir.mkdir(exist_ok=True, parents=True)
            out = out_dir / f.name
            shutil.copy(f, out)

In [116]:
def save_him_files(out_root, files, start):
    with tqdm(list(files.iteritems())) as bar:
        for (band, seg), f in bar:
            out_dir = out_root / start.strftime('%Y%m%dT%H%M') / f'{band:02d}'
            out_dir.mkdir(exist_ok=True, parents=True)
            out = out_dir / f.name
            shutil.copy(f, out)

In [105]:
g16_files = get_or_files(G16_DIR)
g17_files = get_or_files(G17_DIR)
h8_files = get_dat_files(H8_DIR)

In [118]:
g16_fd = get_fd_goes(g16_files, start, end)
g17_fd = get_fd_goes(g17_files, start, end)
h8_fd = get_fd_him(h8_files, start)

In [114]:
save_goes_files(L1B_DIR / 'g16', g16_fd)

100%|██████████| 16/16 [00:09<00:00,  1.64it/s]


In [115]:
save_goes_files(L1B_DIR / 'g17', g17_fd)

100%|██████████| 16/16 [00:09<00:00,  1.74it/s]


In [121]:
save_him_files(L1B_DIR / 'h8', h8_fd, start)

100%|██████████| 160/160 [00:34<00:00,  4.60it/s]
